<a href="https://colab.research.google.com/github/hackerinheels/dresser/blob/main/dresser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a notebook that builds a model to identify a traditional dresser against a contemporary or industrial style dresser.

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
print(tf.__version__)

2.6.0


In [ ]:

from google.colab import files
uploaded = files.upload()


In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/furndata/data.zip"

data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='data', 
                                   untar=True)
data_dir = pathlib.Path(data_dir)

87498752/87488946 [==============================] - 1s 0us/step


In [ ]:
import glob
for name in data_dir.glob("*"):
    print(name)

686


In [ ]:
#@title Colab Shell
%%html
<div id=term_demo></div>
<script src="https://code.jquery.com/jquery-latest.js"></script>
<script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
<link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
<script>
  $('#term_demo').terminal(async function(command) {
      if (command !== '') {
          try {
              let res = await google.colab.kernel.invokeFunction('shell', [command])
              let out = res.data['application/json'][0]
              this.echo(new String(out))
          } catch(e) {
              this.error(new String(e));
          }
      } else {
          this.echo('');
      }
  }, {
      greetings: 'Welcome to Colab Shell',
      name: 'colab_demo',
      height: 250,
      prompt: 'colab > '
  });

In [ ]:
# First run unzip data.tar.gz
!cd /root/.keras/datasets/
!unzip data.tar.gz
print(len(list(data_dir.glob('*/*'))))


In [ ]:
traditional = list(data_dir.glob('*/*'))

In [ ]:
roses = list(data_dir.glob('traditional/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 683 files belonging to 3 classes.
Using 136 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['contemporary', 'industrial', 'traditional']


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
])
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
num_classes = 3

model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

Epoch 1/10
18/18 [==============================] - 23s 1s/step - loss: 1.2949 - accuracy: 0.3711 - val_loss: 1.0844 - val_accuracy: 0.4044
Epoch 2/10
18/18 [==============================] - 18s 995ms/step - loss: 1.0628 - accuracy: 0.4314 - val_loss: 1.0182 - val_accuracy: 0.4044
Epoch 3/10
18/18 [==============================] - 18s 993ms/step - loss: 0.9974 - accuracy: 0.4351 - val_loss: 0.9639 - val_accuracy: 0.4632
Epoch 4/10
18/18 [==============================] - 18s 993ms/step - loss: 0.9311 - accuracy: 0.5448 - val_loss: 0.8807 - val_accuracy: 0.5735
Epoch 5/10
18/18 [==============================] - 18s 993ms/step - loss: 0.8951 - accuracy: 0.5667 - val_loss: 0.8916 - val_accuracy: 0.5662
Epoch 6/10
18/18 [==============================] - 18s 992ms/step - loss: 0.8712 - accuracy: 0.5960 - val_loss: 0.8480 - val_accuracy: 0.6397
Epoch 7/10
18/18 [==============================] - 18s 992ms/step - loss: 0.8314 - accuracy: 0.6015 - val_loss: 0.9298 - val_accuracy: 0.5368
Ep

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

In [ ]:
checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1,
  callbacks=[cp_callback]
)
# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

18/18 [==============================] - 20s 1s/step - loss: 0.3687 - accuracy: 0.8592 - val_loss: 1.4006 - val_accuracy: 0.5662

Epoch 00001: saving model to cp.ckpt


In [ ]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
#loss, acc = model.evaluate(test_images, test_labels, verbose=2)


In [ ]:
#Import libraries
#import pandas
import pandas as pd
#importing numpy
import numpy as np
#importing tensorflow
import tensorflow as tf
#importing keras from tensorflow
from tensorflow import keras
# importing Sequential from keras
from tensorflow.keras.models import Sequential
#importing Dense and Conv2D layers from keras
from tensorflow.keras.layers import Dense,Conv2D

def build_model(hp):
    # create model object
    model = Sequential([
    #adding first convolutional layer    
    layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(28,28,1)),
    #layers.MaxPool2D(),
    # adding second convolutional layer 
    layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        #activation function
        activation='relu'
    ),
    #layers.MaxPool2D(),
    layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_3_filter', min_value=32, max_value=64, step=16),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_3_kernel', values = [3,5]),
        #activation function
        activation='relu'
    ),# adding flatten layer    
    #layers.MaxPool2D(),
    layers.Flatten(),
    # adding dense layer    
    layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # output layer    
    layers.Dense(3, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
train_labels = np.concatenate([y for x, y in train_ds], axis=0)
val_labels = np.concatenate([y for x, y in val_ds], axis=0)


In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 97 kB 4.8 MB/s 


In [ ]:
#importing random search

from kerastuner import RandomSearch
#creating randomsearch object
tuner = RandomSearch(build_model,
                    objective='val_accuracy',
                    max_trials = 5)
# search best parameter
tuner.search(train_ds,train_labels,epochs=3,validation_data=(train_ds,train_labels))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  This is separate from the ipykernel package so we can avoid doing imports until



Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |80                |?                 
conv_1_kernel     |5                 |?                 
conv_2_filter     |64                |?                 
conv_2_kernel     |3                 |?                 
conv_3_filter     |64                |?                 
conv_3_kernel     |5                 |?                 
dense_1_units     |48                |?                 
learning_rate     |0.001             |?                 

Invalid model 0/5


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/ops.py", line 1880, in _create_c_op
    c_op = pywrap_tf_session.TF_FinishOperation(op_desc)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling2d_2/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](Placeholder)' with input shapes: [?,1,1,64].

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/keras_tuner/engine/hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-42-6f5ebd8a1267>", line 54, in build_model
    layers.Dense(3, activation='softmax')
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/training/tracking/base.py", line 530, in _method_wrapper


Invalid model 1/5
Invalid model 2/5


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/ops.py", line 1880, in _create_c_op
    c_op = pywrap_tf_session.TF_FinishOperation(op_desc)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling2d_2/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](Placeholder)' with input shapes: [?,1,1,64].

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/keras_tuner/engine/hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-42-6f5ebd8a1267>", line 54, in build_model
    layers.Dense(3, activation='softmax')
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/training/tracking/base.py", line 530, in _method_wrapper


Invalid model 3/5
Invalid model 4/5


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/ops.py", line 1880, in _create_c_op
    c_op = pywrap_tf_session.TF_FinishOperation(op_desc)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling2d_2/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](Placeholder)' with input shapes: [?,1,1,64].

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/keras_tuner/engine/hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-42-6f5ebd8a1267>", line 54, in build_model
    layers.Dense(3, activation='softmax')
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/training/tracking/base.py", line 530, in _method_wrapper


Invalid model 5/5


RuntimeError: ignored